In [1]:
#Clickhouse db w/ Goldsky
# https://clickhouse.com/docs/en/integrations/python

import requests as r
import pandas as pd
import clickhouse_connect as cc
import os

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import pandas_utils as p
import clickhouse_utils as ch
sys.path.pop()

import time

client = ch.connect_to_clickhouse_db() #Default is OPLabs DB
# client.close()

In [2]:
chain_mappings_list = [
    # {'schema_name': 'zora', 'display_name': 'Zora', 'has_blob_fields': False},
    # {'schema_name': 'pgn', 'display_name': 'Public Goods Network', 'has_blob_fields': False},
    # {'schema_name': 'base', 'display_name': 'Base', 'has_blob_fields': False},
    {'schema_name': 'mode', 'display_name': 'Mode', 'has_blob_fields': False},
    {'schema_name': 'metal', 'display_name': 'Metal', 'has_blob_fields': False},
    {'schema_name': 'fraxtal', 'display_name': 'Fraxtal', 'has_blob_fields': True},
    {'schema_name': 'bob', 'display_name': 'BOB (Build on Bitcoin)', 'has_blob_fields': False},
    # Add more mappings as needed
]
chain_mappings_dict = {item['schema_name']: item['display_name'] for item in chain_mappings_list}

block_time_sec = 2

trailing_days = 9999
max_execution_secs = 3000

In [3]:
sql_directory = "inputs/sql/"

query_names = [
        # Must match the file name in inputs/sql
        "ch_template_alltime_chain_activity"
]

In [4]:
unified_dfs = []
table_name = 'op_ch_allltime_chain_activity'

In [5]:
for qn in query_names:
        for mapping in chain_mappings_list:
                chain_schema = mapping['schema_name']
                display_name = mapping['display_name']
                has_blob_fields = mapping['has_blob_fields']
                # If we can do it programmatically from UI saved queries
                # query = client.get_job(query_name)
                # Read the SQL query from file
                with open(os.path.join(sql_directory, f"{qn}.sql"), "r") as file:
                        query = file.read()
                print(qn + ' - ' + chain_schema)
                table_name = qn

                #Pass in Params to the query
                query = query.replace("@chain_db_name@", chain_schema)
                query = query.replace("@trailing_days@", str(trailing_days))
                query = query.replace("@block_time_sec@", str(block_time_sec))
                query = query.replace("@max_execution_secs@", str(max_execution_secs))

                if ~has_blob_fields:
                        query = query.replace("receipt_l1_blob_base_fee_scalar", 'cast(NULL as Nullable(Float64))')
                        query = query.replace("receipt_l1_blob_base_fee", 'cast(NULL as Nullable(Float64))')
                        query = query.replace("receipt_l1_base_fee_scalar", 'toInt64(NULL)')
                # Execute the query
                result_df = client.query_df(query)
        #         # Write to csv
        #         df.to_csv('outputs/chain_data/' + qn + '.csv', index=False)
        #         # print(df.sample(5))
        #         time.sleep(1)
                
                result_df['chain_raw'] = result_df['chain']
                result_df['chain'] = result_df['chain'].replace(chain_mappings_dict)
                unified_dfs.append(result_df)

        write_df = pd.concat(unified_dfs)
        write_df.to_csv('outputs/chain_data/' + table_name + '.csv', index=False)
        d.write_dune_api_from_pandas(write_df, table_name,table_description = table_name)
        
        # # # Print the results

ch_template_alltime_chain_activity - mode
SELECT *
    , num_blocks * block_time_sec as active_secs_per_day
    
    , l2_num_txs_per_day / num_blocks AS l2_num_txs_per_day_per_block
        , l2_num_txs_per_day / (num_blocks*block_time_sec) AS num_user_txs_per_second
        
    , l2_gas_used / num_blocks AS l2_gas_used_per_block
        , l2_gas_used / (num_blocks*block_time_sec) AS l2_gas_used_per_second
        
    , l2_eth_fees_per_day / num_blocks AS l2_eth_fees_per_block
        , l2_eth_fees_per_day / (num_blocks*block_time_sec) AS l2_eth_fees_per_second
    
--     , CASE WHEN dt = MIN(dt) OVER (PARTITION BY chain, network) THEN 1 ELSE 0 END AS is_initial_day
    
    , chain as chain_name --make name align with registry

FROM (
    SELECT
        DATE_TRUNC('day', toDateTime(block_timestamp)) AS dt,
        chain, network, 2 AS block_time_sec,

        COUNT(*) AS num_raw_txs,
        COUNT(DISTINCT block_number) AS num_blocks,

        SUM(CASE WHEN gas_price = 0 AND to_ad

In [12]:
print(write_df['chain'].unique())

<StringArray>
['Mode', 'Metal', 'Fraxtal', 'BOB']
Length: 4, dtype: string


In [9]:
write_df.sample(5)

,dt,chain,network,block_time_sec,num_raw_txs,num_blocks,l2_num_attr_deposit_txs_per_day,l2_num_user_deposit_txs_per_day,l2_num_txs_per_day,l2_num_success_txs_per_day,...,avg_l1_blob_fee_scalar,active_secs_per_day,l2_num_txs_per_day_per_block,num_user_txs_per_second,l2_gas_used_per_block,l2_gas_used_per_second,l2_eth_fees_per_block,l2_eth_fees_per_second,chain_name,chain_raw
26,2024-01-09,Mode,mainnet,2,43244,43200,43200,1,43,43,...,0.0,86400,0.000995,0.000498,5.125738e+04,25628.691748,6.829244e-07,3.414622e-07,mode,mode
32,2024-04-25,Fraxtal,mainnet,2,48056,43200,43200,39,4813,4717,...,0.0,86400,0.111412,0.055706,1.901066e+05,95053.321516,1.612822e-06,8.064108e-07,fraxtal,fraxtal
8,2024-03-20,Mode,mainnet,2,142682,42174,33015,454,109213,106168,...,0.0,84348,2.589581,1.294791,1.185208e+06,592604.118770,9.114637e-06,4.557319e-06,mode,mode
28,2024-04-19,Metal,mainnet,2,43202,43200,43200,1,1,1,...,0.0,86400,0.000023,0.000012,4.518363e+04,22591.817326,5.946776e-09,2.973388e-09,metal,metal
105,2023-11-20,Mode,mainnet,2,43384,43200,43200,24,160,153,...,0.0,86400,0.003704,0.001852,5.152124e+04,25760.621377,2.102044e-06,1.051022e-06,mode,mode
